In [3]:
'''==图象识别模型CNN、RNN、LSTM、Attention……等 精简解析版=='''
# coding=utf-8
import tensorflow as tf
import os
import numpy as np
#读数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('whale/MinistData', one_hot=True)


Extracting whale/MinistData\train-images-idx3-ubyte.gz
Extracting whale/MinistData\train-labels-idx1-ubyte.gz
Extracting whale/MinistData\t10k-images-idx3-ubyte.gz
Extracting whale/MinistData\t10k-labels-idx1-ubyte.gz


In [7]:
'''
==CNN卷积神经网络 图象识别 精简骨架==
'''

def CNN(FLAGS):
    #输入输出初始化 x y
    inputs = tf.placeholder(dtype=tf.float32,shape=[None,FLAGS.image_width*FLAGS.image_height])
    output = tf.placeholder(dtype=tf.int32, shape=[None, FLAGS.num_classes])
    print("==CNN输入==",inputs)
    #==构造卷积神经网络==
    #==第一层==
    # 1、权重项初始化 w 
    w_1 = tf.Variable(tf.truncated_normal([5,5,1,32],stddev=0.1))
    #Variable tf生成变量
    #truncated_normal 压宿初始化为正态分布值，shape张量（向量加深度）维度  truncate缩短 trun-枝干
    #filter滑动窗口大小 shape 5 5 1 32 高 宽 深RGB灰 输出32个特征图度，stddev标准差
    # 2、偏移量初始化 b
    b_1 = tf.Variable(tf.random_normal([32],stddev=0.1))#初使化为正态分布随机数
    #b_1 = tf.Variable(tf.constant(0.1,shape=[32]))
    #生成常数constant 32个，wx+b输入结果是32个特征图神经元，所以w,b都是输出32维度
    # 3、第一层卷积 conv,用tf的tf.nn.conv2d()，relu激活函数在这一层加，下一步只用来浓缩卷积处理好的特征图
    #取x整理
    x_1 = tf.reshape(inputs,[-1,FLAGS.image_height,FLAGS.image_width,FLAGS.channel])
    print("==CNN输入变形最终一维==",x_1)
    #reshape特征拉平用于计算 wx 使维度匹配
    conv_1 = tf.nn.conv2d(x_1,w_1,strides=[1,1,1,1],padding='SAME')
    #conv_1 = tf.nn.relu(conv_1+b_1)
    conv_1 = tf.nn.relu(tf.nn.bias_add(conv_1,b_1))
    #tf.nn.conv2d(x,w,strides=[1,1,1,1],padding='SAME')+b , SAME不满格时补充
    #【卷积的是wx 滑动计算】 5 5 5 5 filter滑动窗口大小，步长 1 1 1 1第次向右向下走一格，结果+b偏移量，再整体relu
    # 4、第一层池化 把卷积后的特征图 压缩pooling 浓缩代表（特征群取最大值代表用tf.nn.max_pool()）特征图 pool
    pool_1 = tf.nn.max_pool(conv_1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    #ksize = [1,2,2,1]#池化压缩窗口大小2*2 开始的窗口是5*5，strides步长
    #前后两个1一般不管，有一个chanel一个batch 不在这两个维度做池化（只是平面的特征压缩，深度和batch不变 设1）
    #加dropout
    pool_1 = tf.nn.dropout(pool_1,keep_prob=FLAGS.keep_drop)
    
    #==第二层==
    w_2 = tf.Variable(tf.truncated_normal([5,5,32,64],stddev=0.1))
    #这一层的w，图高宽还是5*5，深度不再是一张灰度 而是前面传过来的32个特征图，这次输出64个特征图
    #truncated_normal 生成随机数 不是nn的神经网络的函数
    b_2 = tf.Variable(tf.random_normal([64],stddev=0.1))
    conv_2 = tf.nn.conv2d(pool_1,w_2,strides=[1,1,1,1],padding='SAME')
    conv_2 = tf.nn.relu(tf.nn.bias_add(conv_2,b_2))
    #这里的输入是上一层最后的输出pool后的结果pool_1
    pool_2 = tf.nn.max_pool(conv_2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    #加dropout
    pool_2 = tf.nn.dropout(pool_2,keep_prob=FLAGS.keep_drop)
    
    
    #==全连接层（全连接，即，此层不卷积，全部参与连接wx+b计算最终输出分类连softmax）==
    fc_w_1 = tf.Variable(tf.truncated_normal([7*7*64,1024],stddev=0.1))
    #全连接层的输入是一维的，把前一层池化结果拉平，w也要维度匹配,
    #这里用了固定的7*7 28*28图经过一层池化压缩变14*14(28/2 2*2池化),经第二次压缩变7*7，再*64个特征图，
    #输出是1024维
    fc_b_1 = tf.Variable(tf.random_normal([1024],stddev=0.1))
    #计算本层wx+b 全连接层全参与计算，上一层pool_2做输入，几个特征图要拉平变一维
    flat_pool_2 = tf.reshape(pool_2,[-1,7*7*64])#拉平成只有高宽两元素的，
    #-1表式bachsize不管？还是chanel不动？还是跟据另一维自动变？待查
    #计算wx+b
    fc_1 = tf.nn.relu(tf.add(tf.matmul(flat_pool_2,fc_w_1),fc_b_1))
    #tf.matmul相当于np.dot，矩阵乘，做内积，不是对应位置乘
    
    #==加Dropout（全连接层每个神经元都保留会过拟合，舍弃一些,按比例keep_prob保留）==
    #if train == True:#控制是训练还是测试，训练集dropout
    fc_1 = tf.nn.dropout(fc_1,keep_prob=FLAGS.keep_drop)   
    
    #==连softmax分类器层变成类别概率值==
    fc_w_2 = tf.Variable(tf.truncated_normal([1024,10],stddev=0.1))
    #输入上一个全连接层的输出1024，输出10个分类
    fc_b_2 = tf.Variable(tf.random_normal([10],stddev=0.1))#b匹配的是输入，有多少个结果对应就是多少个wx+b个式子，即多少个b
    #【卷积计算需要输入float型 常数项b也一样，要设初始值为0.0或0.1】
    #【别忘了第二参是shape=】
    #softmax计算
    #_pred = tf.nn.softmax(tf.add(tf.matmul(fc_1,fc_w_2),fc_b_2))
    #【注意：以上不可在外面加softmax，加完直接用tf.nn.softmax_cross_entropy_with_logits算交叉熵损失即可，
    #这个类里面已经整合了softmax】
    _pred = tf.add(tf.matmul(fc_1,fc_w_2),fc_b_2)
    #插播一条：logits log(p/(1-p)) 广义线性模型里的，比值比，多分类，用比值比 logits变成符合线性的多分类
    
    #==以上两层卷积神经网络构造完成
    #以下是算结果的loss，和精度计算==
    
    #==用平均交叉熵计算loss==
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred,labels=output))
    #logits分类结果与labels output y做比较求loss 验证准确率
    
    #==梯度下降优化模型（相当于普通神经网络的反向传播优化的过程），
    #用tf.train.AdamOptimizer 梯度下降Adam优化器Optimizer优化，
    #作用是寻找全局最优点优化算法，引入二次方梯度校正
    #lr学习率==
    train_op = tf.train.AdamOptimizer(learning_rate=FLAGS.lr).minimize(cost)
        
    #==验证准确率==
    _corr=tf.equal(tf.argmax(_pred,1),tf.argmax(output,1))#axis=1比较行中各元素
    #预测值argmax最大索引（最大的概率代表它的分类），与真实最大索引equal相等说明预测正确。
    #算平均损失
    accuracy = tf.reduce_mean(tf.cast(_corr,tf.float32))#cast类型转换 #'float' 换
    
    return train_op,cost,accuracy,inputs,output#返回训练下降优化函数、损失函数、准确率计算函数

In [ ]:
#【以上做了几个优化，有利于提升模型精确度：
#关键处BUG修改：最后一层只需算完add+，即用于 softmax_cross_entropy_with_logits 交叉熵运算，
#不要先在外面加一层softmax】
#还做了以下几个优化
#b，由固定常数变为正态分布随机数random_normal([32],stddev=0.1
#b的+用了tf.nn.bias_add 最后一层用了tf.add，因为维度只有一维了
#epoch和batch_size的调整之类

In [14]:
'''
==RNN循环神经网络 图象识别 精简骨架==
'''
from tensorflow.contrib import rnn
#图片28*28象素用LSTM记忆单元记忆前一时序列相关数据，
#思路：需将完整图切格成分散的序列，依次输入，人为制造前后相继的相关联性

def RNN(FLAGS,_name):
    
     #定义维度
    diminput = FLAGS.image_width #输入维度图宽28
    nsteps = FLAGS.image_height #切分几份序列，也就是图高
    dimhidden = 128 #隐层神经元数定为128
    dimoutput = FLAGS.num_classes #输出10分类
    
    #==第一层，输入计算wx+b 切分序列==
    
    #定义输入输出参数纬度
    inputs = tf.placeholder('float',[None,nsteps*diminput])#shape (图高*宽 [?,28*28])输入不再是完整的图
    output = tf.placeholder('float',[None,dimoutput])
    
    print("==RNN 输入：==",inputs)
    _X = tf.reshape(inputs,[-1,nsteps,diminput])
    #变形shape (图高,宽 [?,28,28])#输入图高宽分开，高要切分成多行序列打散输入
    print("==RNN 输入变形：==",_X)
    _X = tf.transpose(_X,[1,0,2])
    #transpose维度换位置，即把中间28行换到前面 [28,?,28]，变28组，便于后面切分按行输入
    print("==RNN transpose inputs换位==",_X)
    _X = tf.reshape(_X,[-1,diminput])
    #拉平变一维[?,28]，用于wx+b计算，保持图宽，特征列
    print('==RNN inputs reshape变形最终一维==',_X)
    
    w_h = tf.Variable(tf.random_normal([diminput,dimhidden]))#输入层与隐层计算用w 维度 输入28 输出128
    b_h = tf.Variable(tf.random_normal([dimhidden]))#输入层与隐层计算用b 维度 直接加上，与输出128匹配即可
    
    _H = tf.matmul(_X,w_h) + b_h #计算wx+b
    
    #切分
    #上面先整体计算，计算完连接下一层时切分成序列，是为了方便计算
    _Hsplit = tf.split(_H,nsteps,0)#切成28行，依次传入28个序列 0按行切，_H切的对象是上一步的计算结果
    #【切分的目的初衷：是为了让一个整体图变成分散有关联的子图，
    #用RNN的LSTM门学习记忆之前的相关序列信息，
    #最终决策图的分类】
    
    #==第二层，接入RNN LSTM门单元==
    with tf.variable_scope(_name,reuse=tf.AUTO_REUSE):
    #变量参数共享，再次运行时变量不会报已定义BUG,
    #旧版注释
    #with tf.variable_scope(_name) as scope:
        #scope.reuse_variables()
        
        #【1、先处理门控lstm_cell 前一序列的中间结果按保留率保留，tf.nn.rnn_cell.BasicLSTMCell 或rnn.BasicLSTMCell】
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(dimhidden,forget_bias=1.0)#维度尺寸 隐层神经元大小128，forget_bias 保留率，一开始全保留
        #【2、组建RNN神经网络,这里需组合前面门控取得的前一序列中间结果，
        #和上一步计算的wx+b结果 切分后的序列 _Hsplit。用rnn.static_rnn】
        _LSTM_O,_LSTM_S = rnn.static_rnn(lstm_cell,_Hsplit,dtype = tf.float32)
        #输出结果：两部分，本步结果和中间计算结果】
    
    #==第三层，取最终结果==
    w_o = tf.Variable(tf.random_normal([dimhidden,dimoutput]))#[输入隐层维度128，输出10分类]
    w_b = tf.Variable(tf.random_normal([dimoutput]))
    #计算wx+b x为上一步的输出结果_LSTM_O,w 为
    _O_pred = tf.matmul(_LSTM_O[-1],w_o) + w_b
    #[-1]只取最后一次的_LSTM_O即可，前面的已通过每一步LSTM门控保留中间结果累加组合到最后了
    
    #==算平均损失、预测准确率==
    #交叉熵损失（softmax在这里整合了，所以上一步pred结果只需wx+b即可，不用接入softmax了
    #分类值处损失没用均方误差，用了交叉熵，因为）
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_O_pred,labels=output))
    #train_op = tf.train.GradientDescentOptimizer(FLAGS.lr).minimize(cost)
    with tf.variable_scope(_name,reuse=tf.AUTO_REUSE):#参数共享，防止多次运行报错已定义
        train_op = tf.train.AdamOptimizer(FLAGS.lr).minimize(cost)
        #可选优化器：梯度下降优化器tf.train.GradientDescentOptimizer，
        #和tf.train.AdamOptimizer，大概比较Adam精度高些
    correct_preds = tf.equal(tf.argmax(_O_pred,1),tf.argmax(output,1))#预测值最大概率（分类）与真值最大概率（分类）一致说明预测正确
    accuracy = tf.reduce_mean(tf.cast(correct_preds,'float'))
    
    return train_op,cost,accuracy,inputs,output
    

In [20]:
#==训练神经网络图片分类任务==

#训练函数
def train(sess,FLAGS,train_op,cost,accuracy,inputs,output):
    #=开始训练=
    # begin train
    for epoch in range(FLAGS.epochs):  
        print("==epoch: %d/%d"%(epoch,FLAGS.epochs))
        #totalBatch = int(mnist.train.num_examples / FLAGS.batch_size)
        avg_cost = 0 #平均损失
        totalBatch = 10

        for k in range(totalBatch):
            batch_xs,batch_ys = mnist.train.next_batch(FLAGS.batch_size)
            #print("==batch_xs.shape==",batch_xs.shape)
            #print("==batch_ys.shape==",batch_ys.shape)
            #【sess 处理上面 train_op(梯度下降优化迭代器)#logits分类结果,交差，acc精度等计算结果,
            #返回loss值，精确度等
            #并feed tensorflow数据，在这一步实现】
            #_, network, loss, acc = sess.run([train_op, _pred, cost, accuracy], 
            #                                 feed_dict={inputs: batch[0], output: batch[1]})
            #【可以在一起run，也可以分开，如下】：
            feed_dict_v = {inputs: batch_xs, output: batch_ys}
            sess.run(train_op,feed_dict=feed_dict_v)
            avg_cost += sess.run(cost,feed_dict=feed_dict_v)/totalBatch
            acc = sess.run(accuracy,feed_dict=feed_dict_v)
            #【需要返回值的函数就要用sess run一下，为placeholder占位的inputs output赋值
            #精确度和loss的结果是在这里返回的，要用sess.run一下，返回的是相当于函数执行的结果，不能直接取】
        print("batch:%d/%d %d %d"%(k+1,totalBatch,FLAGS.batch_size,mnist.train.num_examples))    
        print('平均损失 : %f 训练准确率 : %f'% (avg_cost, acc))
    #print('测试准确率:', accuracy.eval({inputs: mnist.test.images, output: mnist.test.labels}))

#参数定义
def defineParams():
    #tf.reset_default_graph()#重置图
    #train=True
    FLAGS = tf.app.flags.FLAGS
    #if len(FLAGS) <=4:#【解决二次运行报错定义两次BUG:
    #未定义过时（FLAGS长度为4，里面有4个HELP项）才定义，】
    #放弃，下次运行时直接点工具栏刷新按钮重启服务再运行即可。
    #或像下面这种删除定义的属性再定义
    #tf.flags.FLAGS.__delattr__('image_width')
    #tf.flags.FLAGS.__delattr__('image_height')
    #tf.flags.FLAGS.__delattr__('num_classes')
    #第三种方法：可用with tf.variable_scope(_name) as scope:结合scope.reuse_variables() 变量共享解决
    #参考 https://www.cnblogs.com/hellcat/p/9194110.html
    
    for key in sorted(FLAGS):
        tf.flags.FLAGS.__delattr__(key)#删除定义的所有key,再定义
    
    #【==tensorflow参数定义，类似于类里的属性定义的作用
    #tf三种变量定义与赋值比较:
    #flags：定义的直接给下面用的单一值，可以不定义在参数里，直接用等号定义也可以使用；
    #Variable：是定义一个格式几维的变量组有初始化的值；
    #placeholder：是先占位，再在sess下，feed传值，相当于一个函数传参的作用
    #（它也是先有一个格式，但与Variable不同的是没初始化值，后面传进去的）
    #(Variable初始化的值，因为是一个函数，所以需要run执行下 
    #run的是初始化函数类对象，sess.run(tf.global_variables_initializer())
    #我们自己实例化的类对象需要执行或取值时run我们自己取的对象名字，返回值是相应函数的计算结果)
    #】==
    tf.app.flags.DEFINE_integer('image_width', 28, '图宽')
    tf.app.flags.DEFINE_integer('image_height', 28, '图高')
    tf.app.flags.DEFINE_integer('num_classes', 10, '分几类')
    tf.app.flags.DEFINE_integer('channel',1,'通道RGB 1 灰度图')
    tf.app.flags.DEFINE_float('keep_drop',.7,'dropout保留率70%')
    tf.app.flags.DEFINE_float('lr',0.001,'学习率')
    #tf.app.flags.DEFINE_string('checkpoints','./checkpoints/model.ckpt','模型保存路径')
    tf.app.flags.DEFINE_integer('batch_size',16,'一次训练多少图')
    tf.app.flags.DEFINE_integer('epochs',15,'训练轮数，所有数据跑一遍为一轮')
    
    FLAGS.flag_values_dict()

    #打印参数
    print("\n参数：")
    for key in sorted(FLAGS):
        value = FLAGS[key].value
        print("{}={}".format(key.upper(),value)) 
        
    return FLAGS    

#tensorflow session run处理部分
def runNN(text,FLAGS,train_op,cost,accuracy,inputs,output):
    
    #==tensorflow sess处理执行部分==
    
    sess_config = tf.ConfigProto(
        allow_soft_placement = True,
        log_device_placement = True
    )
    #allow_soft_placement True tf自动选择设备 log_device_placement True 获取tf自动指派的设备并打印
    
    sess = tf.Session(config=sess_config)
    
    with sess.as_default():#创建一个默认会话,#类似的with tf.name_scope是命名空间，防止有name的命名冲突
        
        #实例化保存类
        #saver = tf.train.Saver(max_to_keep=1000)#只保留1000次迭代
        
        #run 变量初始化
        sess.run(tf.global_variables_initializer())
        
        #保存模型,就是sess
        #saver.restore(sess,FLAGS.checkpoints)#checkpoints保存路径
        
        print('\n=='+text+'==\n')
        train(sess,FLAGS,train_op,cost,accuracy,inputs,output)
        
        
#主函数
def main(_):#参数不能为空，要指定一个占位
    #==取参数==
    FLAGS = defineParams()
    #==训练CNN 注意，训练NN不能放在with see.as_default中==
    train_op,cost,accuracy,inputs,output = CNN(FLAGS)
    #==训练RNN LSTM==
    train_op_rnn,cost_rnn,accuracy_rnn,inputs_rnn,output_rnn = RNN(FLAGS,'basic')  
    
    runNN('CNN训练结果',FLAGS,train_op,cost,accuracy,inputs,output)
    runNN('RNN LSTM训练结果',FLAGS,train_op_rnn,cost_rnn,accuracy_rnn,inputs_rnn,output_rnn)
        
if __name__ == '__main__':
    
    tf.app.run()# 执行main函数



参数：
BATCH_SIZE=16
CHANNEL=1
EPOCHS=15
IMAGE_HEIGHT=28
IMAGE_WIDTH=28
KEEP_DROP=0.7
LR=0.001
NUM_CLASSES=10
==CNN输入== Tensor("Placeholder_20:0", shape=(?, 784), dtype=float32)
==CNN输入变形最终一维== Tensor("Reshape_20:0", shape=(?, 28, 28, 1), dtype=float32)
==RNN 输入：== Tensor("Placeholder_22:0", shape=(?, 784), dtype=float32)
==RNN 输入变形：== Tensor("Reshape_22:0", shape=(?, 28, 28), dtype=float32)
==RNN transpose inputs换位== Tensor("transpose_6:0", shape=(28, ?, 28), dtype=float32)
==RNN inputs reshape变形最终一维== Tensor("Reshape_23:0", shape=(?, 28), dtype=float32)

==CNN训练结果==

==epoch: 0/15
batch:10/10 16 55000
平均损失 : 7.572825 训练准确率 : 0.062500
==epoch: 1/15
batch:10/10 16 55000
平均损失 : 2.891717 训练准确率 : 0.375000
==epoch: 2/15
batch:10/10 16 55000
平均损失 : 1.995711 训练准确率 : 0.500000
==epoch: 3/15
batch:10/10 16 55000
平均损失 : 1.712369 训练准确率 : 0.562500
==epoch: 4/15
batch:10/10 16 55000
平均损失 : 1.294055 训练准确率 : 0.437500
==epoch: 5/15
batch:10/10 16 55000
平均损失 : 1.073926 训练准确率 : 0.437500
==epoch: 6/15
batc

SystemExit: 

In [ ]:
'''
==Attention模型==
【持续更新中，请稍假关注】
'''